In [1]:
import cv2
import numpy as np

In [2]:
PATH2DATA = 'C:/Users/abram/Documents/books/Kopytov/FIVES A Fundus Image Dataset for AI-based Vessel Segmentation'

Load image

In [14]:
def get_img():
    img = cv2.imread(PATH2DATA + '/train/Original/1_A.png')
    return cv2.resize(img, (512, 512))

In [19]:
img2draw = get_img()
img = cv2.cvtColor(img2draw, cv2.COLOR_BGR2GRAY)
cv2.imshow('Original', img)
cv2.waitKey(0)

-1

In [6]:
thresh = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,53,10)
cv2.imshow('bin',thresh)
cv2.waitKey(0)

-1

Adaptive threshold

In [24]:
img2draw = get_img()
thresh = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,53,10)
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
cv2.drawContours(img2draw, contours, -1,(255,0,0),2)
cv2.imshow('test',img2draw)
cv2.waitKey(0)

-1

Canny

In [33]:
img2draw = get_img()
blurred = cv2.GaussianBlur(img, (5, 5), 0)
thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,53,13)
edges = cv2.Canny(thresh,100,200)
contours, _ = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
cv2.drawContours(img2draw, contours, -1,(255,0,0),2)
cv2.imshow('canny',img2draw)
cv2.waitKey(0)

-1

In [39]:
img2draw = get_img()
edges = cv2.Canny(img,100,200)
contours, _ = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
cv2.drawContours(img2draw, contours, -1,(255,0,0),2)
cv2.imshow('canny',img2draw)
cv2.waitKey(0)

-1

Sobel

In [13]:
sobel_x = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=3)  # Градиент по оси X
sobel_y = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=3)  # Градиент по оси Y
sobel_combined = cv2.magnitude(sobel_x, sobel_y)
sobel_combined = np.uint8(np.absolute(sobel_combined))

img2draw = img.copy()
thresh = cv2.adaptiveThreshold(sobel_combined,255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,53,10)
contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
cv2.drawContours(img2draw, contours, -1,(255,0,0),2)
cv2.imshow('sobel',img2draw)
cv2.waitKey(0)

-1